In [2]:
#install dependencies
#----------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
sns.set_style('whitegrid')
sns.set_context('notebook')
sns.color_palette("tab10", 10)

from itertools import chain

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [5]:
filename = "../data_processed/csv/wisconsin.csv"
df = pd.read_csv(filename, index_col=0)
df

,sample_code_num,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2.0,1,1,1,2
695,841769,2,1,1,1,2,1.0,1,1,1,2
696,888820,5,10,10,3,7,3.0,8,10,2,4
697,897471,4,8,6,4,3,4.0,10,6,1,4


In [6]:
df_dropped = df.drop(columns=["sample_code_num"])
df_dropped.head()

,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2


In [7]:
#drop null values
#----------------------------------------------
df_dropped = df_dropped.dropna()
df_dropped.shape

(683, 10)

In [8]:
df_dropped.to_csv('../data_processed/csv/wisconsin_clean.csv')

In [63]:
avg_df=df_dropped.groupby('class').mean()
avg_df = avg_df.T
avg_df = avg_df.reset_index()
avg_df = avg_df.rename_axis(None, axis=1)
avg_df = avg_df.rename(columns={'index':'category'})
avg_df



,category,2,4
0,clump_thickness,2.963964,7.188285
1,uniformity_cell_size,1.306306,6.577406
2,uniformity_cell_shape,1.414414,6.560669
3,marginal_adhesion,1.346847,5.585774
4,single_epithelial_cell_size,2.108108,5.326360
5,bare_nuclei,1.346847,7.627615
6,bland_chromatin,2.083333,5.974895
7,normal_nucleoli,1.261261,5.857741
8,mitoses,1.065315,2.602510


In [64]:
median_df=df_dropped.groupby('class').median()
median_df = median_df.T
median_df = median_df.reset_index()
median_df = median_df.rename_axis(None, axis=1)
median_df = median_df.rename(columns={'index':'category'})
median_df

,category,2,4
0,clump_thickness,3.0,8.0
1,uniformity_cell_size,1.0,6.0
2,uniformity_cell_shape,1.0,6.0
3,marginal_adhesion,1.0,5.0
4,single_epithelial_cell_size,2.0,5.0
5,bare_nuclei,1.0,10.0
6,bland_chromatin,2.0,7.0
7,normal_nucleoli,1.0,6.0
8,mitoses,1.0,1.0


In [65]:
var_df=df_dropped.groupby('class').var()
var_df = var_df.T
var_df = var_df.reset_index()
var_df = var_df.rename_axis(None, axis=1)
var_df = var_df.rename(columns={'index':'category'})
var_df

,category,2,4
0,clump_thickness,2.797796,5.943392
1,uniformity_cell_size,0.732150,7.421504
2,uniformity_cell_shape,0.915909,6.600295
3,marginal_adhesion,0.841051,10.218452
4,single_epithelial_cell_size,0.769325,5.968672
5,bare_nuclei,1.387326,9.713688
6,bland_chromatin,1.128480,5.209451
7,normal_nucleoli,0.911272,11.214971
8,mitoses,0.259833,6.576632


In [66]:
stdev_df=df_dropped.groupby('class').std()
stdev_df = stdev_df.T
stdev_df = stdev_df.reset_index()
stdev_df = stdev_df.rename_axis(None, axis=1)
stdev_df = stdev_df.rename(columns={'index':'category'})
stdev_df

,category,2,4
0,clump_thickness,1.672661,2.437907
1,uniformity_cell_size,0.855657,2.724244
2,uniformity_cell_shape,0.957031,2.569104
3,marginal_adhesion,0.917088,3.196631
4,single_epithelial_cell_size,0.877112,2.443087
5,bare_nuclei,1.177848,3.116679
6,bland_chromatin,1.062299,2.282422
7,normal_nucleoli,0.954606,3.348876
8,mitoses,0.509738,2.564495
